# Feb 1, 2025: testing brainrender

In [1]:
import os
import numpy as np
import nibabel as nib
from brainrender import Scene, actor, Atlas
from tqdm import tqdm

import vedo
vedo.settings.default_backend = 'k3d'
from vedo import Volume, show

import ants

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

np.random.seed(args.SEED)

In [3]:
BASE_path = f'{os.environ["HOME"]}/new_mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
os.system(f'mkdir -p {PARCELS_path}')

0

In [4]:
args.source = 'allen'
args.space = 'ccfv2'
args.brain_div = 'whl'
args.num_rois = 120
args.resolution = 25 #200, 25 #um

PARC_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.resolution}'
)
PARC_DESC

'source-allen_space-ccfv2_braindiv-whl_nrois-120_res-25'

In [5]:
TEMP_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_res-{args.resolution}'
)
TEMP_DESC

'source-allen_space-ccfv2_res-25'

In [6]:
temp_file = f'{PARCELS_path}/{TEMP_DESC}_desc-template.nii.gz'
temp_img = nib.load(temp_file)
temp_vol = temp_img.get_fdata()

temp_vol = np.transpose(temp_vol, (2, 1, 0))
temp_vol = temp_vol[::-1, :, :]

In [7]:
parcels_file = f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz'
parcels_img = nib.load(parcels_file)
parcels_vol = np.round(parcels_img.get_fdata(), decimals=0)

parcels_vol = np.transpose(parcels_vol, (2, 1, 0))
parcels_vol = parcels_vol[::-1, :, :]

In [8]:
temp_vol.shape, parcels_vol.shape

((528, 320, 456), (528, 320, 456))

In [9]:
scene = Scene()

In [10]:
def add_vol(scene: Scene, vol: Volume, color='#cccccc', alpha=0.2):
    mesh = vol.isosurface()
    mesh = mesh.smooth(
        niter=1000, 
        pass_band=0.01, 
        edge_angle=15, 
        feature_angle=150,
        boundary=False,
    )
    mesh_actor = actor.Actor(
        mesh, 
        br_class='Volume',
        color=color,
        alpha=alpha,
    )
    scene.add(mesh_actor)
    return scene

In [11]:
# vol = Volume(temp_vol > 1, origin=[0]*3, spacing=[args.resolution]*3)
# scene = add_vol(scene, vol, color='#cccccc', alpha=0.2)

In [12]:
def add_roi(scene, roi, color='lightblue', alpha=0.5):
    roi_vol = (parcels_vol == roi)
    vol = Volume(roi_vol, origin=[0]*3, spacing=[args.resolution]*3)
    scene = add_vol(scene, vol, color=color, alpha=0.5)
    return scene

In [ ]:
for roi in tqdm(np.unique(parcels_vol)[1:30]):
    color = [np.random.random(), np.random.random(), np.random.random()]
    scene = add_roi(scene, roi=roi, color=color, alpha=0.1)
# scene = add_roi(scene, roi=13, color='red', alpha=0.5)
# scene = add_roi(scene, roi=26, color='green', alpha=0.5)
# scene = add_roi(scene, roi=39, color='pink', alpha=0.5)

  7%|▋         | 2/29 [00:05<01:24,  3.15s/it]

In [ ]:
# _ = scene.actors[0].alpha(0)

In [ ]:
scene.export(f'test.html')

Cannot run function _get_inset in a jupyter notebook Try setting the correct backend before creating your scene:

from vedo import embedWindow                                                                                       
embedWindow(None)                                                                                                  

Your scene is ready for rendering, use:

from vedo import show                                                                                              
vedo.show(*scene.renderables)                                                                                      

/home/govindas/miniconda3/envs/gt/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


The brainrender scene has been exported for web. The results are saved at test.html

'test.html'